In [25]:
import os
import pandas as pd
import re

In [5]:
asc14 = os.path.join('data', 'ACS_14_1YR_S1701_clean.csv')
brf14 = os.path.join('data', 'BRFSS_2014_Overall_DUI_n_SNUFF.csv')

In [15]:
asc14_DF = pd.read_csv(asc14)
asc14_slice = asc14_DF[['GEO.display-label', 'HC01_EST_VC01', 'HC02_EST_VC04']].rename(columns={'GEO.display-label':'state', 'HC01_EST_VC01': 'total(est.)poverty_pop.', 'HC02_EST_VC04': 'total(est.)poverty_under18'})

In [100]:
asc14_slice.to_csv('data/acs14.csv', index=False)
asc14_slice

state  total(est.)poverty_pop.  total(est.)poverty_under18
0                Alabama                  4727549                      299717
1                 Alaska                   717723                       27741
2                Arizona                  6573369                      398754
3               Arkansas                  2882856                      179355
4             California                 38060228                     2007029
5               Colorado                  5237149                      185393
6            Connecticut                  3485629                      109751
7               Delaware                   909611                       34966
8   District of Columbia                   624327                       29202
9                Florida                 19470210                      933030
10               Georgia                  9823859                      638868
11                Hawaii                  1379804                       43019
12                 Idaho                  1603083                       78394
13              Illinois                 12571848                      582212
14               Indiana                  6391460                      326069
15                  Iowa                  3004857                      105497
16                Kansas                  2821431                      122934
17              Kentucky                  4275359                      253973
18             Louisiana                  4518525                      302101
19                 Maine                  1294009                       46689
20              Maryland                  5835377                      168388
21         Massachusetts                  6509672                      202976
22              Michigan                  9686787                      481270
23             Minnesota                  5332552                      183274
24           Mississippi                  2895470                      208199
25              Missouri                  5878658                      279206
26               Montana                   997952                       39377
27              Nebraska                  1828121                       72556
28                Nevada                  2800768                      141644
29         New Hampshire                  1287504                       32849
30            New Jersey                  8762159                      310524
31            New Mexico                  2045525                      142903
32              New York                 19228524                      918641
33        North Carolina                  9691794                      539098
34          North Dakota                   712548                       23260
35                  Ohio                 11276403                      580925
36              Oklahoma                  3759517                      204455
37                Oregon                  3893475                      175974
38          Pennsylvania                 12371337                      501858
39          Rhode Island                  1015655                       40822
40        South Carolina                  4691589                      283825
41          South Dakota                   824373                       36240
42             Tennessee                  6386663                      378596
43                 Texas                 26340247                     1702958
44                  Utah                  2897762                      115973
45               Vermont                   601819                       18090
46              Virginia                  8080300                      283702
47            Washington                  6927889                      265946
48         West Virginia                  1792949                       89528
49             Wisconsin                  5603486                      228692
50               Wyoming                 

In [34]:
brf14_DF = pd.read_csv(brf14)
question1 = 'Respondents who have reported having driven after drinking too much (variable calculated from one or more BRFSS questions)'
question2 = 'Do you currently use chewing tobacco, snuff, or snus?'
brf14_DUI = brf14_DF.loc[brf14_DF['Question'] == question1]
brf14_SNUFF = brf14_DF.loc[brf14_DF['Question'] == question2]

In [101]:
acceptance = brf14_DUI.loc[brf14_DUI['Response']=='Have driven after having too much to drink'][['Locationdesc', 'Sample_Size']].rename(columns={'Locationdesc': 'state', 'Sample_Size': 'DUI_admission'})
denial = brf14_DUI.loc[brf14_DUI['Response']=='Have not driven after having too much to drink'][['Locationdesc', 'Sample_Size']].rename(columns={'Locationdesc': 'state', 'Sample_Size': 'DUI_denial'})
dui14 = pd.merge(acceptance, denial, on='state')[0:51]
dui14['DUI_admission'] = dui14['DUI_admission'].astype(int)
dui14['DUI_denial'] = dui14['DUI_denial'].str.replace(",","").astype(int)
dui14.to_csv('data/brf14.csv', index=False)
dui14

state  DUI_admission  DUI_denial
0                Alabama             73        2852
1                 Alaska             58        2345
2                Arizona            108        6938
3               Arkansas             39        1601
4             California            141        3818
5               Colorado            198        7083
6            Connecticut            117        4464
7               Delaware             53        2041
8   District of Columbia             54        2261
9                Florida            113        4856
10               Georgia             36        2543
11                Hawaii            136        3336
12                 Idaho             65        2243
13              Illinois             68        2716
14               Indiana            125        4759
15                  Iowa            157        4029
16                Kansas            155        6295
17              Kentucky             77        3019
18             Louisiana            104        2911
19                 Maine            106        4903
20              Maryland            126        6191
21         Massachusetts            219        8452
22              Michigan            158        4426
23             Minnesota            276        9528
24           Mississippi             41        1275
25              Missouri             72        2687
26               Montana            151        3800
27              Nebraska            461       11607
28                Nevada             63        1915
29         New Hampshire             53        3451
30            New Jersey            146        6746
31            New Mexico             66        3602
32              New York             58        3480
33        North Carolina             63        3016
34          North Dakota            176        4121
35                  Ohio            165        4744
36              Oklahoma             58        2990
37                Oregon             67        2863
38          Pennsylvania            164        5262
39          Rhode Island            101        3614
40        South Carolina            117        4549
41          South Dakota            133        3715
42             Tennessee             41        1658
43                 Texas            187        6608
44                  Utah             91        3946
45               Vermont             90        3880
46              Virginia             99        4408
47            Washington            154        5669
48         West Virginia             27        1920
49             Wisconsin            153        3967
50               Wyoming             61        2780

In [107]:
df = pd.merge(dui14, asc14_slice, on='state', how='outer').rename(columns={'DUI_admission': 'admit', 'DUI_denial':'deny', 'total(est.)poverty_pop.': 'poverty(total)', 'total(est.)poverty_under18' : 'poverty(<18)'})
df.to_csv('data/data.csv', index=False)
df

state  admit   deny  poverty(total)  poverty(<18)
0                Alabama     73   2852         4727549        299717
1                 Alaska     58   2345          717723         27741
2                Arizona    108   6938         6573369        398754
3               Arkansas     39   1601         2882856        179355
4             California    141   3818        38060228       2007029
5               Colorado    198   7083         5237149        185393
6            Connecticut    117   4464         3485629        109751
7               Delaware     53   2041          909611         34966
8   District of Columbia     54   2261          624327         29202
9                Florida    113   4856        19470210        933030
10               Georgia     36   2543         9823859        638868
11                Hawaii    136   3336         1379804         43019
12                 Idaho     65   2243         1603083         78394
13              Illinois     68   2716        12571848        582212
14               Indiana    125   4759         6391460        326069
15                  Iowa    157   4029         3004857        105497
16                Kansas    155   6295         2821431        122934
17              Kentucky     77   3019         4275359        253973
18             Louisiana    104   2911         4518525        302101
19                 Maine    106   4903         1294009         46689
20              Maryland    126   6191         5835377        168388
21         Massachusetts    219   8452         6509672        202976
22              Michigan    158   4426         9686787        481270
23             Minnesota    276   9528         5332552        183274
24           Mississippi     41   1275         2895470        208199
25              Missouri     72   2687         5878658        279206
26               Montana    151   3800          997952         39377
27              Nebraska    461  11607         1828121         72556
28                Nevada     63   1915         2800768        141644
29         New Hampshire     53   3451         1287504         32849
30            New Jersey    146   6746         8762159        310524
31            New Mexico     66   3602         2045525        142903
32              New York     58   3480        19228524        918641
33        North Carolina     63   3016         9691794        539098
34          North Dakota    176   4121          712548         23260
35                  Ohio    165   4744        11276403        580925
36              Oklahoma     58   2990         3759517        204455
37                Oregon     67   2863         3893475        175974
38          Pennsylvania    164   5262        12371337        501858
39          Rhode Island    101   3614         1015655         40822
40        South Carolina    117   4549         4691589        283825
41          South Dakota    133   3715          824373         36240
42             Tennessee     41   1658         6386663        378596
43                 Texas    187   6608        26340247       1702958
44                  Utah     91   3946         2897762        115973
45               Vermont     90   3880          601819         18090
46              Virginia     99   4408         8080300        283702
47            Washington    154   5669         6927889        265946
48         West Virginia     27   1920         1792949         89528
49             Wisconsin    153   3967         5603486        228692
50               Wyoming     61   2780          570149         16262

In [104]:
brf14_SNUFF

Year Locationabbr                                Locationdesc  \
2    2014           AL                                     Alabama   
3    2014           AL                                     Alabama   
4    2014           AL                                     Alabama   
7    2014           AK                                      Alaska   
8    2014           AK                                      Alaska   
9    2014           AK                                      Alaska   
12   2014           AZ                                     Arizona   
13   2014           AZ                                     Arizona   
14   2014           AZ                                     Arizona   
17   2014           AR                                    Arkansas   
18   2014           AR                                    Arkansas   
19   2014           AR                                    Arkansas   
22   2014           CA                                  California   
23   2014           CA                                  California   
24   2014           CA                                  California   
27   2014           CO                                    Colorado   
28   2014           CO                                    Colorado   
29   2014           CO                                    Colorado   
32   2014           CT                                 Connecticut   
33   2014           CT                                 Connecticut   
34   2014           CT                                 Connecticut   
37   2014           DE                                    Delaware   
38   2014           DE                                    Delaware   
39   2014           DE                                    Delaware   
42   2014           DC                        District of Columbia   
43   2014           DC                        District of Columbia   
44   2014           DC                        District of Columbia   
47   2014           FL                                     Florida   
48   2014           FL                                     Florida   
49   2014           FL                                     Florida   
..    ...          ...                                         ...   
227  2014           VT                                     Vermont   
228  2014           VT                                     Vermont   
229  2014           VT                                     Vermont   
232  2014           VA                                    Virginia   
233  2014           VA                                    Virginia   
234  2014           VA                                    Virginia   
237  2014           WA                                  Washington   
238  2014           WA                                  Washington   
239  2014           WA                                  Washington   
242  2014           WV                               West Virginia   
243  2014           WV                               West Virginia   
244  2014           WV                               West Virginia   
247  2014           WI                                   Wisconsin   
248  2014           WI                                   Wisconsin   
249  2014           WI                                   Wisconsin   
252  2014           WY                                     Wyoming   
253  2014           WY                                     Wyoming   
254  2014           WY                                     Wyoming   
257  2014           GU                                        Guam   
258  2014           GU                                        Guam   
259  2014           GU                                        Guam   
262  2014           PR                                 Puerto Rico   
263  2014           PR                                 Puerto Rico   
264  2014           PR                                 Puerto Rico   
267  2014           UW               All States and DC (median) **   
268  2014      